In [1]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
import time
import trajectory_msgs.msg 
import tf
import tf.transformations
from math import pi
from std_msgs.msg import String,Empty,UInt16
from moveit_commander.conversions import pose_to_list
import numpy as np
import helper
import math
from origami_plan import *

In [2]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('dual_arm_origami', anonymous=True)
listener = tf.TransformListener()
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
rospy.sleep(2)
group_name1 = "hong_arm"
group1 = moveit_commander.MoveGroupCommander(group_name1)
group_name2 = "kong_arm"
group2 = moveit_commander.MoveGroupCommander(group_name2)
group_name3 = "hong_hand"
group3 = moveit_commander.MoveGroupCommander(group_name3)
group_name4 = "kong_hand"
group4 = moveit_commander.MoveGroupCommander(group_name4)
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path',
                                               moveit_msgs.msg.DisplayTrajectory,
                                               queue_size=20)

In [3]:
def addCollisionObjects():
    global box_pose1
    global box_name1
    box_pose1 = geometry_msgs.msg.PoseStamped()
    box_pose1.header.frame_id = "world"
    box_pose1.pose.orientation.w = 1.0
    box_pose1.pose.position.x = 0.105
    box_pose1.pose.position.y = 0.1485
    box_pose1.pose.position.z = 0.71
    box_name1 = 'box1'
    scene.add_box(box_name1, box_pose1, (0.01, 0.01, 0.01))
    global box_pose2
    global box_name2
    box_pose2 = geometry_msgs.msg.PoseStamped()
    box_pose2.header.frame_id = "world"
    box_pose2.pose.orientation.w = 1.0
    box_pose2.pose.position.x = -0.105
    box_pose2.pose.position.y = 0.1485
    box_pose2.pose.position.z = 0.71
    box_name2 = 'box2'
    scene.add_box(box_name2, box_pose2, (0.01, 0.01, 0.01))
    global box_pose3
    global box_name3
    box_pose3 = geometry_msgs.msg.PoseStamped()
    box_pose3.header.frame_id = "world"
    box_pose3.pose.orientation.w = 1.0
    box_pose3.pose.position.x = -0.105
    box_pose3.pose.position.y = -0.1485
    box_pose3.pose.position.z = 0.71
    box_name3 = 'box3'
    scene.add_box(box_name3, box_pose3, (0.01, 0.01, 0.01))
    global box_pose4
    global box_name4
    box_pose4 = geometry_msgs.msg.PoseStamped()
    box_pose4.header.frame_id = "world"
    box_pose4.pose.orientation.w = 1.0
    box_pose4.pose.position.x = 0.105
    box_pose4.pose.position.y = -0.1485
    box_pose4.pose.position.z = 0.71
    box_name4 = 'box4'
    scene.add_box(box_name4, box_pose4, (0.01, 0.01, 0.01))
    global paper_pose
    global paper_name
    paper_pose = geometry_msgs.msg.PoseStamped()
    paper_pose.header.frame_id = "world"
    paper_pose.pose.orientation.w = 1.0
    paper_pose.pose.position.x = 0
    paper_pose.pose.position.y = 0
    paper_pose.pose.position.z = 0.706
    paper_name = 'paper'
    scene.add_box(paper_name, paper_pose, (0.21, 0.297, 0.002))   
    global crease_pose1
    global crease_name1
    crease_pose1 = geometry_msgs.msg.PoseStamped()
    crease_pose1.header.frame_id = "world"
    crease_pose1.pose.orientation.x = 0
    crease_pose1.pose.orientation.y = 0
    crease_pose1.pose.orientation.z = -0.38268
    crease_pose1.pose.orientation.w = 0.9238777
    crease_pose1.pose.position.x = -0.03075
    crease_pose1.pose.position.y = -0.07425
    crease_pose1.pose.position.z = 0.7065
    crease_name1 = 'crease1'
    global crease_pose2
    global crease_name2
    crease_pose2 = geometry_msgs.msg.PoseStamped()
    crease_pose2.header.frame_id = "world"
    crease_pose2.pose.orientation.x = 0
    crease_pose2.pose.orientation.y = 0
    crease_pose2.pose.orientation.z = 0.38268
    crease_pose2.pose.orientation.w = 0.9238777
    crease_pose2.pose.position.x = -0.03075
    crease_pose2.pose.position.y = 0.07425
    crease_pose2.pose.position.z = 0.7065
    crease_name2 = 'crease2'    
    global crease_pose3
    global crease_name3
    crease_pose3 = geometry_msgs.msg.PoseStamped()
    crease_pose3.header.frame_id = "world"
    crease_pose3.pose.orientation.w = 1
    crease_pose3.pose.position.x = 0
    crease_pose3.pose.position.y = -0.07425
    crease_pose3.pose.position.z = 0.7065
    crease_name3 = 'crease3' 
    global wall_pose1
    global wall_name1
    wall_pose1 = geometry_msgs.msg.PoseStamped()
    wall_pose1.header.frame_id = "world"
    wall_pose1.pose.orientation.w = 1
    wall_pose1.pose.position.x = 0
    wall_pose1.pose.position.y = 0.75
    wall_pose1.pose.position.z = 1
    wall_name1 = "wall1"
    global wall_pose2
    global wall_name2
    wall_pose2 = geometry_msgs.msg.PoseStamped()
    wall_pose2.header.frame_id = "world"
    wall_pose2.pose.orientation.w = 1
    wall_pose2.pose.position.x = 0
    wall_pose2.pose.position.y = -0.75
    wall_pose2.pose.position.z = 1
    wall_name2 = "wall2"  
#     global wall_pose3
#     global wall_name3
#     wall_pose3 = geometry_msgs.msg.PoseStamped()
#     wall_pose3.header.frame_id = "world"
#     wall_pose3.pose.orientation.w = 1
#     wall_pose3.pose.position.x = 0
#     wall_pose3.pose.position.y = 0
#     wall_pose3.pose.position.z = 1.85
#     wall_name3 = "wall3"
#     scene.add_box(wall_name3,wall_pose3,(1.2,0.04,1.3))

In [4]:
def robkong_go_to_home(times,vel):
    if times == 1:
        joint_values = [-1.75, -1.95, -1.34, -1.4, 1.7, 0.22]
    elif times == 2:
        joint_values = [-2.03, -1.1, 1.93, -2.42, -1.5, 1.23]
    group2.set_joint_value_target(joint_values)
    plan = group2.plan()
    new_traj = scale_trajectory_speed(plan, vel)
    group2.execute(new_traj)
    group2.stop()
    group2.clear_pose_targets()

def robhong_go_to_home(times,vel):
    if times == 1:
        joint_values = [0.96, -1.07, 1.37, -1.31, -1.7, 0.32]
    elif times == 2:
        joint_values = [-2.04,-1.28,2.33,-2.69,-1.56,0.84]
    group1.set_joint_value_target(joint_values)
    plan = group1.plan()
    new_traj = scale_trajectory_speed(plan, vel)
    group1.execute(new_traj)
    group1.stop()
    group1.clear_pose_targets()

In [5]:
def move_relateive_waypoints(dx,dy,dz,vel,robot_arm):
    '''
    This function is used for moving position of tool0 link of robot_arm.
    dx, dy, dz are relative distance between target position and initial position.
    '''
    if robot_arm == "robhong":
        group = group1
    elif robot_arm == "robkong":
        group = group2
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0        
    waypoints = []
    waypoints.append(group.get_current_pose().pose)
    wpose = copy.deepcopy(group.get_current_pose().pose)
    wpose.position.x += dx
    wpose.position.y += dy
    wpose.position.z += dz
    waypoints.append(copy.deepcopy(wpose))
    (plan, fraction) = group.compute_cartesian_path(waypoints, 0.01, 0.0)
    if fraction == 1:
        # if plan successful, execute and return 1
        new_traj = scale_trajectory_speed(plan,vel)
        group.execute(new_traj)
        print "Move relative waypoints to the target position succeed!"
        return 1
    else:
        # else return 0
        print "Move relative waypoints to the target position failed..."
        return 0

In [6]:
def new_move_waypoints(target_position,source_frame,vel,robot_arm):
    '''
    This function is used for moving position of source_frame link of robot_arm. 2 steps: calculation and move.
    target_pos: position in world frame. position of source_frame.
    '''
    if robot_arm == "robhong":
        group = group1
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0 
    
    #step1: transformation, calculate the move distance between target_position and current source_frame position
    listener.waitForTransform("world",source_frame, rospy.Time(), rospy.Duration(4.0))
    (trans1,rot1) = listener.lookupTransform("world",source_frame, rospy.Time(0))
    listener.waitForTransform( "world",target_frame,rospy.Time(), rospy.Duration(4.0))
    (trans2,rot2) = listener.lookupTransform("world",target_frame,rospy.Time(0))
    trans_target_source = np.subtract(trans1,trans2).tolist()
    current_pos = group.get_current_pose()
    delta = [target_position[0]-current_pos.pose.position.x-trans_target_source[0],
            target_position[1]-current_pos.pose.position.y-trans_target_source[1],
            target_position[2]-current_pos.pose.position.z-trans_target_source[2]]

    #step2: move waypoints in a straight line to the target_position
    waypoints = []
    waypoints.append(current_pos.pose)
    wpose = copy.deepcopy(current_pos.pose)
    wpose.position.x += delta[0]
    wpose.position.y += delta[1]
    wpose.position.z += delta[2]
    waypoints.append(copy.deepcopy(wpose))
    (plan, fraction) = group.compute_cartesian_path(waypoints, 0.01, 0.0)
    if fraction == 1:
        # if plan successful, execute and return 1
        new_traj = scale_trajectory_speed(plan,vel)
        group.execute(new_traj) 
        group.clear_pose_targets()
        print "Move waypoints to the target position succeed!"
        return 1
    else:
        print "Move waypoints to the target position failed..."
        return 0
    
def new_rot_waypoints(target_quat,source_frame,vel,robot_arm):
    '''
    This function is used for rotating orientation of source_frame link of robot_arm. 2 steps: calculation and move.
    target_quat: quat in world frame. quat of source_frame.
    '''
    if robot_arm == "robhong":
        group = group1
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0 
    #step1: calculate the goal_quat of ${robot_arm}_tool0
    listener.waitForTransform(source_frame,target_frame, rospy.Time(), rospy.Duration(4.0))
    (trans,rot) = listener.lookupTransform(source_frame, target_frame, rospy.Time(0))
    goal_quat = tf.transformations.quaternion_multiply(target_quat,rot) #goal_quat
    current_pos =group.get_current_pose().pose
    initial_quat = [current_pos.orientation.x,current_pos.orientation.y,
                    current_pos.orientation.z,current_pos.orientation.w] #initial_quat

    #step2: rotate waypoints to the target orientation
    waypoints = []
    for t in range(0, 4):
        quat_waypoints = tf.transformations.quaternion_slerp(initial_quat,goal_quat,t/3)
        current_pos.orientation.x = quat_waypoints[0]
        current_pos.orientation.y = quat_waypoints[1]
        current_pos.orientation.z = quat_waypoints[2]
        current_pos.orientation.w = quat_waypoints[3]
        waypoints.append(copy.deepcopy(current_pos))
    (plan, fraction) = group.compute_cartesian_path(waypoints, 0.01, 0)
    if fraction == 1:
        new_traj = scale_trajectory_speed(plan,vel)
        group.execute(new_traj)
        group.clear_pose_targets()
        print "Rotate waypoints to the target orientation succeed!"
        return 1
    else:
        print "Rotate waypoints to the target orientation failed..."
        return 0

def new_move_target(target_pose,source_frame, vel, robot_arm):
    if robot_arm == "robhong":
        group = group1
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0 
    #step1: transformation, calculate the move distance between target_pos and source_frame
    listener.waitForTransform("world",source_frame, rospy.Time(), rospy.Duration(4.0))
    (trans1,rot1) = listener.lookupTransform("world",source_frame, rospy.Time(0))
    listener.waitForTransform( "world",target_frame,rospy.Time(), rospy.Duration(4.0))
    (trans2,rot2) = listener.lookupTransform("world",target_frame,rospy.Time(0))
    trans_target_source = np.subtract(trans1,trans2).tolist()
    current_pos = group.get_current_pose()
    delta = [target_pose[0]-trans_target_source[0],
            target_pose[1]-trans_target_source[1],
            target_pose[2]-trans_target_source[2]]
    #step2: transformation: calculate the goal_quat of ${robot_arm}_tool0
    listener.waitForTransform(source_frame,target_frame, rospy.Time(), rospy.Duration(4.0))
    (trans,rot) = listener.lookupTransform(source_frame, target_frame, rospy.Time(0))
    target_quat = [target_pose[3],target_pose[4],target_pose[5],target_pose[6]]
    goal_quat = tf.transformations.quaternion_multiply(target_quat,rot) #goal_quat
    #step3: move to the target pose
    pose_goal = geometry_msgs.msg.Pose()
    pose_goal.orientation.x = goal_quat[0]
    pose_goal.orientation.y = goal_quat[1]
    pose_goal.orientation.z = goal_quat[2]
    pose_goal.orientation.w = goal_quat[3]
    pose_goal.position.x = delta[0]
    pose_goal.position.y = delta[1]
    pose_goal.position.z = delta[2]
    plan0 = group.set_pose_target(pose_goal)
    plan = group.plan(plan0)
    if len(plan.joint_trajectory.points) > 0 :
        new_traj = scale_trajectory_speed(plan,vel)
        group.execute(plan)
        print "Move targets to the target pose succeed!"
        group.clear_pose_targets()
        return 1
    else:
        print "Move targets to the target pose failed..."
        return 0
    
def scale_trajectory_speed(traj, scale):
    new_traj = moveit_msgs.msg.RobotTrajectory()
    new_traj.joint_trajectory = traj.joint_trajectory
    n_joints = len(traj.joint_trajectory.joint_names)
    n_points = len(traj.joint_trajectory.points)
    points = list(traj.joint_trajectory.points)

    for i in range(n_points):
        point = trajectory_msgs.msg.JointTrajectoryPoint()
        point.time_from_start = traj.joint_trajectory.points[i].time_from_start / scale
        point.velocities = list(traj.joint_trajectory.points[i].velocities)
        point.accelerations = list(traj.joint_trajectory.points[i].accelerations)
        point.positions = traj.joint_trajectory.points[i].positions

        for j in range(n_joints):
            point.velocities[j] = point.velocities[j] * scale
            point.accelerations[j] = point.accelerations[j] * scale * scale
        points[i] = point

    new_traj.joint_trajectory.points = points
    return new_traj

In [7]:
def move_autonomously(target_pose,source_frame,vel,robot_arm):
    '''
    This function is used for autonomously moving.
    It involves 3 move functions: new_move_waypoints(), new_rot_waypoints(), move_target().
    The simple logic of this function is to try to implement new_rot_waypoints() and new_move_waypoints() first,
    if the above two move functions failed, try to implement move_target() instead.
    '''
    if robot_arm == "robhong":
        group = group1
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0        
    target_position = [target_pose[0],target_pose[1],target_pose[2]]
    target_quat = [target_pose[3],target_pose[4],target_pose[5],target_pose[6]]

    is_rot_waypoints = new_rot_waypoints(target_quat,source_frame,vel,robot_arm)

    if is_rot_waypoints == 1:
        rospy.sleep(1)
        is_move_waypoints = new_move_waypoints(target_position,source_frame,vel,robot_arm)

        if is_move_waypoints == 1:
            rospy.sleep(1)
            return 1

        elif is_move_waypoints == 0:
            rospy.sleep(1)
            is_move_target = new_move_target(target_pose,source_frame, vel,robot_arm)

            if is_move_target == 1:
                rospy.sleep(1)
                return 1

            elif is_move_target == 0:
                print "all moving methods failed..."
                return 0
    
    elif is_rot_waypoints == 0: 
        rospy.sleep(1)
        is_move_waypoints = new_move_waypoints(target_position,source_frame,vel,robot_arm)
        
        if is_move_waypoints == 1:    
            rospy.sleep(1)
            is_rot_waypoints = new_rot_waypoints(target_quat,source_frame,vel,robot_arm)

            if is_rot_waypoints == 1:
                rospy.sleep(1)
                return 1
            
            elif is_rot_waypoints == 0:
                rospy.sleep(1)
                is_move_target = new_move_target(target_pose,source_frame, vel,robot_arm)

                if is_move_target == 1:
                    rospy.sleep(1)
                    return 1

                elif is_move_target == 0:
                    print "all moving methods failed..." 
                    return 0   
                
        elif is_move_waypoints == 0:
            rospy.sleep(1)
            is_move_target = new_move_target(target_pose,source_frame, vel,robot_arm)

            if is_move_target == 1:
                rospy.sleep(1)
                return 1

            elif is_move_target == 0:
                print "all moving methods failed..." 
                return 0

In [8]:
def tilt(point, axis, angle, velocity, robot_arm):
    '''
    Parameters:
        point (list): 3-D coordinate of point in rotation axis
        axis (list): 3-D vector of rotation axis (right-hand rule)
        angle (double): angle of tilting
        velocity (double): robot velocity between 0 and 1
    '''
    if robot_arm == "robhong":
        group = group1
    elif robot_arm == "robkong":
        group = group2
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0  
    # Normalize axis vector
    axis = axis / np.linalg.norm(axis)
    pose_target = group.get_current_pose().pose
    pos_initial = [pose_target.position.x, pose_target.position.y, pose_target.position.z]
    ori_initial = [pose_target.orientation.x, pose_target.orientation.y, pose_target.orientation.z, pose_target.orientation.w]

    # Tilt center point. Closest point from tcp to axis line
    center = np.add(point, np.dot(np.subtract(pos_initial, point), axis)*axis)

    # Closest distance from tcp to axis line
    radius = np.linalg.norm(np.subtract(center, pos_initial))

    # Pair of orthogonal vectors in tilt plane
    v1 =  -np.subtract(np.add(center, np.dot(np.subtract(pos_initial, center), axis)*axis), pos_initial)
    v1 = v1/np.linalg.norm(v1)
    v2 = np.cross(axis, v1)

    # Interpolate orientation poses via quaternion slerp
    q = helper.axis_angle2quaternion(axis, angle)
    ori_target = tf.transformations.quaternion_multiply(q, ori_initial)
    ori_waypoints = helper.slerp(ori_initial, ori_target, np.arange(1.0/angle , 1.0+1.0/angle, 1.0/angle))

    waypoints = []
    for t in range(1, angle + 1):
        circle = np.add(center, radius*(math.cos(math.radians(t)))*v1 + radius*(math.sin(math.radians(t)))*v2)
        pose_target.position.x = circle[0]
        pose_target.position.y = circle[1]
        pose_target.position.z = circle[2]
        pose_target.orientation.x = ori_waypoints[t-1][0]
        pose_target.orientation.y = ori_waypoints[t-1][1]
        pose_target.orientation.z = ori_waypoints[t-1][2]
        pose_target.orientation.w = ori_waypoints[t-1][3]
        waypoints.append(copy.deepcopy(pose_target))
    (plan, fraction) = group.compute_cartesian_path(waypoints, 0.01, 0) # waypoints, resolution=1cm, jump_threshold)
    plan = scale_trajectory_speed(plan,velocity)
    group.execute(plan)

In [9]:
def fix_paper(fix_point,fix_axis,robot_arm):
    '''
    This function is  used for fixing paper.
    steps: calculate taget pose; move to the target pose.
    orientation: x axis of source_frame will be aligned with fix_axis.
    position: origin of source_frame will be located at fix_point
    '''
    if robot_arm == "robhong":
        group = group1
        source_frame = "rigid_tip_link1"
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        source_frame = "rigid_tip_link2"
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0
    #step1: calculate the target pose for the source_frame
    target_pose = []
    # a): target position
    target_pose = target_pose + fix_point
    target_pose[2] = target_pose[2] + 0.02 #this is to ensure no collision between the source_frame and the table
    # b): target quat  
    fix_axis = np.array(fix_axis) / np.linalg.norm(np.array(fix_axis))
    fix_axis = fix_axis.tolist()
    x_axis = fix_axis
    z_axis = [0,0,-1]
    y_axis = np.cross(z_axis, x_axis).tolist()
    x_axis.append(0)
    y_axis.append(0)
    z_axis.append(0)
    target_rot_mat = np.array([x_axis,
                     y_axis,
                     z_axis,
                     [0,0,0,1]]).transpose()
    target_rot_mat = target_rot_mat.tolist()
    target_quat = tf.transformations.quaternion_from_matrix(target_rot_mat).tolist()
    
    target_pose = target_pose + target_quat
    
    #step2: move to the target pose
    move_autonomously(target_pose,source_frame,0.4,robot_arm)

In [10]:
def flex_flip(paper_edge_point,paper_edge_axis,robot_arm):
    '''
    This function is used for flex-flip.
    steps: calculate target pose; move to the target pose; additional rotate for closing gripper.
    orientation: angle between y axis of source_frame and paper_edge_axis is pi/4 (+:counterclockwise),
                 x axis of source_frame is [0,0,-1] in world frame.
    position: origin of source_frame will be located at paper_edge_point
    '''
    if robot_arm == "robhong":
        group = group1
        source_frame = "soft_gripper_hong"
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        source_frame = "soft_gripper_kong"
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0
    #step1: calculate the target pose for the source_frame
    target_pose = []
    # a): target position
    target_pose = target_pose + paper_edge_point
    target_pose[2] = target_pose[2] + 0.05 #this is to ensure no collision between the source_frame and the table
    # b): target quat    
    paper_edge_axis = paper_edge_axis / np.linalg.norm(paper_edge_axis)
    paper_edge_axis = paper_edge_axis.tolist()
    x_axis = [0,0,-1]
    z_axis = paper_edge_axis
    y_axis = np.cross(z_axis,x_axis).tolist()
    z_axis = paper_edge_axis
    x_axis.append(0)
    y_axis.append(0)
    z_axis.append(0)
    rot_mat_temp = np.array([x_axis,
                     y_axis,
                     z_axis,
                     [0,0,0,1]]).transpose()
    rot_mat_temp = rot_mat_temp.tolist()
    rot_mat = tf.transformations.rotation_matrix(pi/4,(1,0,0))
    target_rot_mat = np.dot(rot_mat_temp,rot_mat).tolist()
    target_quat = tf.transformations.quaternion_from_matrix(target_rot_mat).tolist()
    
    target_pose = target_pose + target_quat
    
    #step2: move autonomosly to the target pose
    move_autonomously(target_pose,source_frame,0.4,robot_arm)
    
    #step3: rotate around z axis of source_frame, and around the origin of source_frame
    listener.waitForTransform("world",source_frame, rospy.Time(), rospy.Duration(4.0))
    (trans,rot) = listener.lookupTransform("world",source_frame, rospy.Time(0))
    center = trans[0:3] #center locates at origin of source_frame
    phi = 15
    rot = tf.transformations.quaternion_matrix(rot).tolist()
    axis = np.dot(rot,[0,0,-1,1]).tolist()[0:3] #z_axis of soft_gripper_$(robot_arm) frame
    tilt(center,axis,phi,0.5,robot_arm)

In [11]:
def fold(crease_point,crease_axis,fold_angle,move_dist,robot_arm,vel=0.4):
    '''
    This function is used for folding. 
    steps: rotate around the crease line, move towards the crease line.
    '''
    #step1: rotate around the crease line
    tilt(crease_point,crease_axis,fold_angle,0.5,robot_arm)
    #step2: move in the direction that vertival to the crease line
    x_axis = [0,0,-1]
    crease_axis = crease_axis / np.linalg.norm(crease_axis)
    direction = np.cross(x_axis,crease_axis)
    direction = direction*move_dist
    move_relateive_waypoints(direction[0],direction[1],direction[2],0.4,robot_arm)

In [12]:
def scooping(scooping_point,scooping_axis,robot_arm,scooping_angle=pi/4,scooping_dis=0.04):
    '''
    This function is used for scooping paper.
    steps: calculate target pose; move to the target pose; move in the scooping direction to scoop paper.
    Orientation: y axis of source_frame will be aligned with scooping_axis, scooping_angle (rad) is the angle between 
        ground and rigid finger of robot_arm.
    Position: origin of source_frame will be located at scooping point.
    Direction: according to right-hand rule, cross product of ([0,0,-1],scooping_axis) in world
    ''' 
    if robot_arm == "robhong":
        group = group1
        source_frame = "rigid_tip_link1"
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        source_frame = "rigid_tip_link2"
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0
    #step1: calculate the target pose for the source_frame
    target_pose = []
    # a): target position
    target_pose = target_pose + scooping_point
    target_pose[2] = target_pose[2] + 0.02 #this is to ensure no collision between the source_frame and the table
    # b): target quat
    scooping_axis = scooping_axis / np.linalg.norm(scooping_axis)
    scooping_axis = scooping_axis.tolist()    
    x_axis = [0,0,-1]
    z_axis = np.cross(x_axis, scooping_axis).tolist()
    y_axis = scooping_axis
    x_axis.append(0)
    y_axis.append(0)
    z_axis.append(0)

    rot_mat_temp = np.array([x_axis,
                     y_axis,
                     z_axis,
                     [0,0,0,1]]).transpose()
    rot_mat_temp = rot_mat_temp.tolist()
    #tilt_angle: rad, i.e. pi/4
    rot_mat = tf.transformations.rotation_matrix(scooping_angle,(0,1,0))
    target_rot_mat = np.dot(rot_mat_temp,rot_mat).tolist()
    target_quat = tf.transformations.quaternion_from_matrix(target_rot_mat).tolist()
    
    target_pose = target_pose + target_quat
    
    #step2: move autonomously to the target pose
    move_autonomously(target_pose,source_frame,0.4,robot_arm)
    
    # step3: move towards scooping direction
    direction = np.array(z_axis[0:3])
    direction = direction*scooping_dis
    move_relateive_waypoints(direction[0],direction[1],direction[2],0.4,robot_arm)

In [25]:
def make_crease(paper_edge_point1,paper_edge_point2,n,robot_arm):
    '''
    This function is used for making a crease.
    steps: caluculate the target pose; move to the target pose; move along the crease line and make crease.
    Orientation: y axis of source_frame will be aligned with [point2 -> point1]
    Position: origin of source_frame will be located at point2
    '''
    if robot_arm == "robhong":
        group = group1
        source_frame = "rigid_tip_link1"
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        source_frame = "rigid_tip_link2"
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0
    #step1: calculate target pose for the source_frame
    target_pose = []
    # a): target position
    target_pose = target_pose + paper_edge_point2
    target_pose[2] = target_pose[2]  #to ensure no collision between the ground and gripper
    # b): target quat
    paper_axis = np.subtract(paper_edge_point1,paper_edge_point2).tolist()
    paper_axis = paper_axis / np.linalg.norm(paper_axis)
    paper_axis = paper_axis.tolist()
    x_axis = [0,0,-1]
    z_axis = np.cross(x_axis, paper_axis).tolist()
    y_axis = paper_axis
    x_axis.append(0)
    y_axis.append(0)
    z_axis.append(0)

    rot_mat_temp = np.array([x_axis,
                     y_axis,
                     z_axis,
                     [0,0,0,1]]).transpose()
    rot_mat_temp = rot_mat_temp.tolist()
    rot_mat = tf.transformations.rotation_matrix(pi/4,(0,1,0))
    target_rot_mat = np.dot(rot_mat_temp,rot_mat).tolist()
    target_quat = tf.transformations.quaternion_from_matrix(target_rot_mat).tolist()
    
    target_pose = target_pose + target_quat
    
    #step2: move autonomously to the target pose
    move_autonomously(target_pose, source_frame, 0.4, robot_arm)
#     new_rot_waypoints(target_quat, source_frame, 0.4, robot_arm)
#     new_move_waypoints(paper_edge_point2, source_frame, 0.1, robot_arm)
    
    #step3: move along the crease axis 
    interval = np.subtract(paper_edge_point1,paper_edge_point2) / n
    direction = np.array(z_axis[0:3])
    direction = direction*0.02
    for i in range(n):
        move_relateive_waypoints(direction[0],direction[1],direction[2],0.4,robot_arm)
        rospy.sleep(1)
        move_relateive_waypoints(-direction[0],-direction[1],-direction[2],0.4,robot_arm) 
        rospy.sleep(1)
        move_relateive_waypoints(interval[0],interval[1],interval[2],0.4,robot_arm)
    move_relateive_waypoints(direction[0],direction[1],direction[2],0.4,robot_arm)
    rospy.sleep(1)
    move_relateive_waypoints(-direction[0],-direction[1],-direction[2],0.4,robot_arm)    

In [32]:
# scene.add_box(crease_name1, crease_pose1, (0.21, 0.01, 0.005))
# scene.add_box(crease_name2, crease_pose2, (0.21, 0.01, 0.005))
scene.remove_world_object(crease_name1)
# scene.remove_world_object(crease_name2)
# scene.remove_world_object(crease_name3)
# scene.remove_world_object(wall_name1)
# scene.remove_world_object(wall_name2)
# scene.add_box(crease_name3, crease_pose3, (0.21, 0.01, 0.005))
# robkong_go_to_home(2)
# robhong_go_to_home(2)
# fix_paper(point1,point2,"robkong")
# flex_flip(point4,edge4,"robhong")
# fold(point3,crease_axis2,30,0.1, "robhong")
# move_waypoints(0.2,0.1,0,0.4,"robkong")
# scene.add_box(crease_name2, crease_pose2, (0.21, 0.01, 0.005))
# make_crease(crease_point3,crease_point1,"robkong")
# scene.add_box(crease_name2, crease_pose2, (0.21, 0.01, 0.005))
# fix_paper(point1,point2,"robhong")
# flex_flip(point4,edge4,"robkong")
# fold(point3,crease_axis2,30,0.05, "robkong")

In [14]:
# We can get the name of the reference frame for this robot:
planning_frame = group1.get_planning_frame()
print "============ Reference frame1: %s" % planning_frame
eef_link = group1.get_end_effector_link()
print "============ End effector1: %s" % eef_link
planning_frame = group2.get_planning_frame()
print "============ Reference frame2: %s" % planning_frame
eef_link = group2.get_end_effector_link()
print "============ End effector2: %s" % eef_link
group_names = robot.get_group_names()
print "============ Robot Groups:", robot.get_group_names()

============ Reference frame1: /world
============ End effector1: hong_wrist_3_link
============ Reference frame2: /world
============ End effector2: kong_wrist_3_link
============ Robot Groups: ['hong_arm', 'hong_hand', 'kong_arm', 'kong_hand']


In [15]:
#_____INITIALIZATION_____#
addCollisionObjects()
rospy.sleep(1)
point1 = [box_pose1.pose.position.x,box_pose1.pose.position.y,box_pose1.pose.position.z]
point2 = [box_pose2.pose.position.x,box_pose2.pose.position.y,box_pose2.pose.position.z]
point3 = [box_pose3.pose.position.x,box_pose3.pose.position.y,box_pose3.pose.position.z]
point4 = [box_pose4.pose.position.x,box_pose4.pose.position.y,box_pose4.pose.position.z]
edge1 = np.subtract(point1,point2).tolist()
edge2 = np.subtract(point2,point3).tolist()
edge3 = np.subtract(point3,point4).tolist()
edge4 = np.subtract(point4,point1).tolist()
crease_axis1 = [-0.7071068,0.7071068,0]
crease_axis2 = [0.7071068,-0.7071068,0]
crease_axis3 = [-1,0,0]
crease_point1 = [-0.105,0,0.76]
crease_point2 = [0.0435,-0.1485,0.76]
crease_point3 = [0.0435,0.1485,0.76]
crease_point4 = [0.105,-0.07425,0.76]
crease_point5 = [-0.015375,-0.07425,0.76]

In [27]:
#_____INITIALIZATION______#
robkong_go_to_home(1,0.4)
robhong_go_to_home(1,0.4)
scene.add_box(wall_name1,wall_pose1,(1.2,0.04,2))
scene.add_box(wall_name2,wall_pose2,(1.2,0.04,2)) 

In [40]:
path1 = "/home/liwei/Desktop/origami_data/paper_configuration/state2.obj"
path2 = "/home/liwei/Desktop/origami_data/paper_configuration/state3.obj"
state1 = State(path1)
state2 = State(path2)
compare = CompareState(state1,state2)

In [41]:
a = compare.grasp_point
b = compare.grasp_line
c = compare.grasp_method
d = compare.fix_point
e = compare.fix_line
f = compare.GP_target_position
g = compare.crease_line
h = compare.crease_point
print "a",a,type(a)
print "b",b,type(b)
print "c",c,type(c)
print "d",d,type(d)
print "e",e,type(e)
print "f",f,type(f)
print "g",g,type(g)
print "h",h,type(h)

a [-43.5 148.5   0. ] <type 'numpy.ndarray'>
b [ 61.5 297.    0. ] <type 'numpy.ndarray'>
c scooping <type 'str'>
d [[ 105.  -148.5    0. ]] <type 'numpy.ndarray'>
e [[   0.    135.75    0.  ]
 [ -67.87  117.56    0.  ]
 [-117.56   67.88    0.  ]
 [-135.75    0.      0.  ]] <type 'numpy.ndarray'>
f [-43.5   0.1   0.2] <type 'numpy.ndarray'>
g [135.75   0.     0.  ] <type 'numpy.ndarray'>
h [[-105.     74.25    0.  ]
 [  30.75   74.25    0.  ]] <type 'numpy.ndarray'>


In [42]:
def ConfigurationAction(compare):
    #data preparation
    compare.DataFormatinMoveit()
    #step1: fix paper
#     fix_point = compare.fix_point[0]/1000
#     fix_point[2] = fix_point[2]+0.71
#     fix_point = fix_point.tolist()
#     fix_line = compare.fix_line[0].tolist()
    point = compare.fix_point[0]
    line = compare.fix_line[0]
    fix_paper(point,line,"robhong")
    #step2: grasp paper
    point = compare.grasp_point
    line = compare.grasp_line
    if compare.grasp_method == "flex flip":
        flex_flip(point,line,"robkong")
    elif compare.grasp_method == "scooping":
        scooping(point,line,"robkong")
    #step3: fold
    point = compare.crease_point[0]
    line = compare.crease_line
    fold(point,line,60,0.08,"robkong")
    #step4: make a crease
    point1 = compare.crease_point[1]
    point2 = compare.crease_point[0]
    make_crease(point1,point2,2,"robhong")

In [43]:
ConfigurationAction(compare)

Rotate waypoints to the target orientation succeed!
Move waypoints to the target position succeed!
Rotate waypoints to the target orientation failed...
Move waypoints to the target position succeed!
Rotate waypoints to the target orientation succeed!
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
Rotate waypoints to the target orientation succeed!
Move waypoints to the target position failed...
Move targets to the target pose succeed!
Move relative waypoints to the target position failed...
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!


In [26]:
pointa = compare.crease_point[1]
pointb = compare.crease_point[0]
make_crease(pointa,pointb,2,"robhong")

Rotate waypoints to the target orientation succeed!
Move waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position failed...
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position failed...
Move relative waypoints to the target position succeed!
Move relative waypoints to the target position succeed!
